In [1]:
!nvidia-smi

Fri Dec 12 14:50:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.02              Driver Version: 581.42         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 Ti     On  |   00000000:01:00.0  On |                  N/A |
| 34%   55C    P0            141W /  300W |    1960MiB /  16303MiB |     50%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Small

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mesolitica/nanot5-small-malaysian-cased")
model = AutoModelForSeq2SeqLM.from_pretrained("mesolitica/nanot5-small-malaysian-cased")

In [3]:
model.config

T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.0,
  "dtype": "float32",
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 32128
}

In [4]:
import torch

device = "cuda" 
model = model.to(device)
model.eval()

TRANSLATE_PREFIX = "terjemah ke Inggeris: "

def translate(text, max_new_tokens=64):
    prompt = TRANSLATE_PREFIX + text
    inputs = tokenizer(prompt, return_tensors="pt")
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            no_repeat_ngram_size=3,
            repetition_penalty=1.1,
            early_stopping=True,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_sentences = [
    "Hari ini saya pergi ke pasar",
    "Saya suka makan nasi lemak",
    "Dia sedang membaca buku di perpustakaan",
]

for sent in test_sentences:
    eng = translate(sent)
    print(f"Malay: {sent}\nEnglish: {eng}\n")

Malay: Hari ini saya pergi ke pasar
English:  ke Inggeris ke Inggeris Ke Saya Hari Hari Saya Hari Ini Saya Saya Hari Saya Hari Saya: Sayater: Saya Saya Saya Saya Saya saya hari ini saya pergi hari ini hari Ini Saya Hari Saya Saya Hari ini Saya Hari Saya Hari Saya Saya Saya: Saya

Malay: Saya suka makan nasi lemak
English:  makan nasi Inggeris: Inggeris: Terterterter Inggeris: : makan: Saya: Saya:: Saya makan makan makan: terjemahterjemahanterter: Inggeris: Inggeristerter Inggeris: ( Inggeris: K Inggeris:

Malay: Dia sedang membaca buku di perpustakaan
English: : Dia sedang membaca buku Dia sedang::ter: Dia sedang sedang membaca dan sedang membaca di perpustakaan di perpustakaan dan Dia sedang Membaca buku di dalam buku di bilik itu sedang membaca Buku di bilik di bilik Dia sedang dan membaca buku di perpustakaan: Dia di sedang membaca::



In [6]:
inputs = tokenizer("This is a test.", return_tensors="pt").to(device)
if "token_type_ids" in inputs:
    inputs.pop("token_type_ids")
out = model.generate(**inputs, max_new_tokens=30)
print(tokenizer.decode(out[0]))


<pad><extra_id_99>.<extra_id_85>.This is a test.<extra_id_84>.<extra_id_83>..................


# Base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mesolitica/nanot5-base-malaysian-cased")
model = AutoModelForSeq2SeqLM.from_pretrained("mesolitica/nanot5-base-malaysian-cased")

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

TRANSLATE_PREFIX = "terjemah ke Inggeris: "

def translate(text, max_new_tokens=64):
    prompt = TRANSLATE_PREFIX + text
    inputs = tokenizer(prompt, return_tensors="pt")
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            no_repeat_ngram_size=3,
            repetition_penalty=1.1,
            early_stopping=True,
        )
    outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return outputs

test_sentences = [
    "Hari ini saya pergi ke pasar",
    "Saya suka makan nasi lemak",
    "Dia sedang membaca buku di perpustakaan",
]

for sent in test_sentences:
    eng = translate(sent)
    print(f"Malay: {sent}\nEnglish: {eng}\n")

Malay: Hari ini saya pergi ke pasar
English:  ini saya pergi ke pasar. Saya pergiterjemahan ke Inggeris: Inggeris: Saya pergi ke pergi Inggeris saya pergi Hari saya pergi saya pergi Inggeris Inggeris Inggeris Saya pergi Saya pergi Inggeris Inggeris ini saya pergi saya pergi Inggeris Inggeris Inggeris Inggeris

Malay: Saya suka makan nasi lemak
English: : ke Inggeris: Saya suka makan nasi lemak: Inggeris: Inggeris: English: Inggeris Saya Saya sukajemah ke Inggeris: Saya suka: Inggeris Saya suka Saya suka Inggeris: Englishterjemahan ke Inggeris nasi lemak nasi lemak Inggeris: Saya

Malay: Dia sedang membaca buku di perpustakaan
English:  Dia sedang membaca buku membaca buku di perpustakaan perpustakaan. Inggeris: Inggeris: di perpustakaan Inggeris Inggeris Inggeris ke Inggeris: Dia sedang menulis buku di Perpustakaan: Dia sedang Membaca bukuterjemahan ke Inggeris di perpustakaan membaca buku di perpustakaan Dia sedang Inggeris Inggeris membaca buku

